## FourSquare API on PA state parks

In [103]:
import numpy as np
import pandas as pd
import requests
import re

from geopy.geocoders import Nominatim 
import folium

In [2]:
# load the list of PA state parks
df_park = pd.read_csv('state_park_PA.csv')
df_park.shape

(121, 7)

In [244]:
# get fourSquare config ids
fsqr_config = pd.read_csv('foursquare_config.txt', header=None)
CLIENT_ID = fsqr_config.iloc[1][0]
CLIENT_SECRET = fsqr_config.iloc[3][0]
VERSION = '20200905' 
LIMIT = 500

In [4]:
# convert an address into latitude and longitude values
geolocator = Nominatim(user_agent="pa_explorer")

latitude = np.empty(df_park.shape[0])
longitude = np.empty(df_park.shape[0])
latitude[:] = np.nan
longitude[:] = np.nan

for ind in range(df_park.shape[0]):
    address = '{}, PA'.format(df_park['Park name'][ind])

    location = geolocator.geocode(address)
    if location:
        latitude[ind] = location.latitude
        longitude[ind] = location.longitude
    print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude[ind], longitude[ind]))
    
np.sum(np.isnan(longitude))


The geograpical coordinate of Allegheny Islands State Park, PA are 40.5371096, -79.82333755703746.
The geograpical coordinate of Archbald Pothole State Park, PA are 41.50989920000001, -75.56838568277055.
The geograpical coordinate of Bald Eagle State Park, PA are 41.0417311, -77.6030495.
The geograpical coordinate of Beltzville State Park, PA are 40.8684893, -75.60191225614028.
The geograpical coordinate of Bendigo State Park, PA are 41.530494399999995, -78.6278593808558.
The geograpical coordinate of Benjamin Rush State Park, PA are 40.11494625, -74.97665930932364.
The geograpical coordinate of Big Pocono State Park, PA are 41.0424311, -75.35193912576602.
The geograpical coordinate of Big Spring State Forest Picnic Area, PA are nan, nan.
The geograpical coordinate of Black Moshannon State Park, PA are 40.90129, -78.06456571768157.
The geograpical coordinate of Blue Knob State Park, PA are 40.2786873, -78.5811281.
The geograpical coordinate of Boyd Big Tree Preserve Conservation Area, 

The geograpical coordinate of Pymatuning State Park, PA are 41.59256695, -80.50988858272981.
The geograpical coordinate of R. B. Winter State Park, PA are 40.9941725, -77.1893037.
The geograpical coordinate of Raccoon Creek State Park, PA are 40.51761045000001, -80.45803128032529.
The geograpical coordinate of Ralph Stover State Park, PA are 40.434521700000005, -75.09927468131067.
The geograpical coordinate of Ravensburg State Park, PA are 41.1100727, -77.243025.
The geograpical coordinate of Reeds Gap State Park, PA are 40.7217364, -77.4752722.
The geograpical coordinate of Ricketts Glen State Park, PA are 41.33284675, -76.27929838443805.
The geograpical coordinate of Ridley Creek State Park, PA are 39.9559436, -75.4496427.
The geograpical coordinate of Ryerson Station State Park, PA are 39.88515845, -80.44362884537824.
The geograpical coordinate of S. B. Elliott State Park, PA are 41.1130063, -78.5258953.
The geograpical coordinate of Salt Springs State Park, PA are 41.91037285, -75.

4

In [7]:
df_park['latitude'] = latitude
df_park['longitude'] = longitude

In [28]:
# add missing park latitude and longitude to the dataframe

# Big Spring State Forest Picnic Area; [40.262894, -77.658821]
df_park.loc[df_park['Park name']=='Big Spring State Forest Picnic Area','latitude'] = 40.262894
df_park.loc[df_park['Park name']=='Big Spring State Forest Picnic Area','longitude'] = -77.658821

# Cowans Gap State Park; [39.990880, -77.927953]
df_park.loc[df_park['Park name']=='Cowans Gap State Park','latitude'] = 39.990880
df_park.loc[df_park['Park name']=='Cowans Gap State Park','longitude'] = -77.927953

# French Creek State Park, PA; [40.214996, -75.789494]
df_park.loc[df_park['Park name']=='French Creek State Park','latitude'] = 40.214996
df_park.loc[df_park['Park name']=='French Creek State Park','longitude'] = -75.789494

# Kings Gap Environmental Education and Training Center [40.093944, -77.267937]
df_park.loc[df_park['Park name']=='Kings Gap Environmental Education and Training Center','latitude'] = 40.093944
df_park.loc[df_park['Park name']=='Kings Gap Environmental Education and Training Center','longitude'] = -77.267937


In [30]:
df_park.to_pickle('df_park.pkl')  
# df_park = pd.read_pickle('df_park.pkl')

In [39]:
df_park

,Park name,County or counties,Area in acres (ha),Date founded,Stream(s) and / or lake(s),Remarks,Image,latitude,longitude
0,Allegheny Islands State Park,Allegheny County,43 acres (17 ha),1980,Allegheny River,Three alluvial islands near Pittsburgh with no...,NaN,40.537110,-79.823338
1,Archbald Pothole State Park,Lackawanna County,150 acres (61 ha),1964,None,"One of world's largest potholes, 38 ft (12 m) ...",NaN,41.509899,-75.568386
2,Bald Eagle State Park,Centre County,"5,900 acres (2,388 ha)",1971,"Bald Eagle Creek, Foster Joseph Sayers Reservoir","1,730 acre (700 ha) U.S. Army Corps of Enginee...",NaN,41.041731,-77.603049
3,Beltzville State Park,Carbon County,"2,973 acres (1,203 ha)",1972,"Pohopoco Creek, Beltzville Lake",U.S. Army Corps of Engineers lake is 949 acres...,NaN,40.868489,-75.601912
4,Bendigo State Park,Elk County,100 acres (40 ha),1959,East Branch Clarion River,"Only 20 acres (8.1 ha) developed, name a corru...",NaN,41.530494,-78.627859
...,...,...,...,...,...,...,...,...,...
116,Washington Crossing Historic Park,Bucks County,500 acres (202 ha),2016,Delaware River,Site of George Washington's crossing of the De...,NaN,40.297607,-74.876276
117,Whipple Dam State Park,Huntingdon County,256 acres (104 ha),1928,Whipple Lake,"There was a camp for Boy Scouts, Girl Scouts, ...",NaN,40.686769,-77.862838
118,White Clay Creek Preserve,Chester County,"1,255 acres (508 ha)",1984,White Clay Creek,"Park was donated by DuPont to preserve ""divers...",NaN,39.743574,-75.769126
119,Worlds End State Park,Sullivan County,780 acres (316 ha),1932,Loyalsock Creek,"A ""Must See Park"" known for trout fishing, whi...",NaN,41.461818,-76.576093


#### show these parks on map

In [228]:
# Visualize neighborhoods on a map with the coordinates
address = 'Centre County, PA'
location = geolocator.geocode(address)
latitude_pa = location.latitude
longitude_pa = location.longitude

In [229]:
# create map of Toronto using latitude and longitude values
map_pa = folium.Map(location=[latitude_pa, longitude_pa], zoom_start=7.5)

# add markers to map
for lat, lng, park, remarks in zip(df_park['latitude'], df_park['longitude'], df_park['Park name'], df_park['Remarks']):
    if ~np.isnan(lat):
        # clean up remarks - remove reference index on wiki
        remarks = re.sub(r'\[[^()]*\]', '', remarks)
        label = "{}: {}".format(park, remarks)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_pa)  
    else:
        print('Missing: {}.'.format(park))
    
map_pa

### nearby venues for each state park

In [310]:
# test API for exploration: 
# `section`: topPicks (a mix of recommendations generated without a query from the user). 
#            OR food, drinks, coffee, shops, arts, outdoors, sights, trending, 
#               nextVenues (venues frequently visited after a given venue), 
# lat = float(df_park.loc[df_park['Park name']=='Cherry Springs State Park', 'latitude'])
# lng = float(df_park.loc[df_park['Park name']=='Cherry Springs State Park', 'longitude'])
parkId = 22
lat = df_park.loc[parkId,'latitude']
lng = df_park.loc[parkId,'longitude']
radius = 10000 # meters, max 100,000 meters; if 25000 m (about half hour drive at 30 mile/h)
SECTION = ''
# create the API request URL for venue recommendation
# "ll" can be substitude by the option "near" with park names directly [automatically resolve geolocate, maybe less reliable]
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT,
    SECTION)

# make the GET request
results = requests.get(url).json()#["response"]['groups'][0]['items']

In [311]:
print('total venues: ' + str(len(results['response']['groups'][0]['items'])))
for i in range(len(results['response']['groups'][0]['items'])):
    print(results['response']['groups'][0]['items'][i]['venue']['name'] + ': ' + 
         results['response']['groups'][0]['items'][i]['venue']['categories'][0]['name'])

total venues: 19
Mountain House Bar & Grill: American Restaurant
Milky Way: American Restaurant
Dollar General: Discount Store
Dollar General: Discount Store
Family Dollar: Discount Store
Save-A-Lot: Grocery Store
Sheetz: Gas Station
Cowans Gap State Park: State / Provincial Park
Giant Food: Grocery Store
gods country: Mountain
The New Fort Restaurant: American Restaurant
Keystone Country Store: Outdoor Supply Store
Milkyway Diner: Diner
Particularly Lumpy Mountain: Mountain
Tower Ridge Inn Bar & Grill: Restaurant
Scrub Ridge inn: American Restaurant
Tuscarora Mountain Summit: Scenic Lookout
The Lodge: Diner
Mackey's Glass Service: Automotive Shop


#### display the venues nearby on map

In [313]:
# map of a park with its nearby venues
map_park = folium.Map(location=[lat, lng], zoom_start=12)
label = "{}: {}".format(df_park['Park name'][parkId], df_park['Remarks'][parkId])
label = folium.Popup(label, parse_html=True)
folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#d63855',
        fill_opacity=0.7,
        parse_html=False).add_to(map_park) 

# add markers to map
for i in range(len(results['response']['groups'][0]['items'])):
    venue_name = results['response']['groups'][0]['items'][i]['venue']['name']
    venue_categ = results['response']['groups'][0]['items'][i]['venue']['categories'][0]['name']
    lat_v = results['response']['groups'][0]['items'][i]['venue']['location']['lat']
    lng_v = results['response']['groups'][0]['items'][i]['venue']['location']['lng']
    
    label = "{} ({})".format(venue_name, venue_categ)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat_v, lng_v],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_park)  
    
map_park

#### Search for the state park venue_id

In [314]:
categId = '5bae9231bedf3950379f89d0' # category id for "State / Provincial Park"
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng,
    categId,
    radius, 
    LIMIT)
venue_park_results = requests.get(url).json()

In [315]:
venue_park_results['response']['venues'][0]

{'id': '4c1e6471fcf8c9b6f72fad0b',
 'name': 'Cowans Gap State Park',
 'location': {'lat': 39.995228113803066,
  'lng': -77.92241152166469,
  'labeledLatLngs': [{'label': 'display',
    'lat': 39.995228113803066,
    'lng': -77.92241152166469}],
  'distance': 676,
  'postalCode': '17233',
  'cc': 'US',
  'city': 'Mc Connellsburg',
  'state': 'PA',
  'country': 'United States',
  'formattedAddress': ['Mc Connellsburg, PA 17233', 'United States']},
 'categories': [{'id': '5bae9231bedf3950379f89d0',
   'name': 'State / Provincial Park',
   'pluralName': 'State / Provincial Parks',
   'shortName': 'State / Provincial Park',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/default_',
    'suffix': '.png'},
   'primary': True}],
 'referralId': 'v-1599801244',
 'hasPerk': False}

#### Get some details about the state park (Premium endpoint)

In [316]:
VENUE_ID = venue_park_results['response']['venues'][0]['id']
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(VENUE_ID, CLIENT_ID, CLIENT_SECRET, VERSION)
venue_results = requests.get(url).json()
venue_results

{'meta': {'code': 200, 'requestId': '5f5b06873dac7251ebcf7929'},
 'response': {'venue': {'id': '4c1e6471fcf8c9b6f72fad0b',
   'name': 'Cowans Gap State Park',
   'contact': {},
   'location': {'lat': 39.995228113803066,
    'lng': -77.92241152166469,
    'labeledLatLngs': [{'label': 'display',
      'lat': 39.995228113803066,
      'lng': -77.92241152166469}],
    'postalCode': '17233',
    'cc': 'US',
    'city': 'Mc Connellsburg',
    'state': 'PA',
    'country': 'United States',
    'formattedAddress': ['Mc Connellsburg, PA 17233', 'United States']},
   'canonicalUrl': 'https://foursquare.com/v/cowans-gap-state-park/4c1e6471fcf8c9b6f72fad0b',
   'categories': [{'id': '5bae9231bedf3950379f89d0',
     'name': 'State / Provincial Park',
     'pluralName': 'State / Provincial Parks',
     'shortName': 'State / Provincial Park',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/default_',
      'suffix': '.png'},
     'primary': True},
    {'id': '4bf58dd8d4

In [317]:
venue_results['response']['venue'].keys()

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'dislike', 'ok', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'popular', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])

In [318]:
venue_results['response']['venue']['rating']

KeyError: 'rating'

In [319]:
venue_results['response']['venue']['likes']

{'count': 3,
 'groups': [{'type': 'others',
   'count': 3,
   'items': [{'id': '49786126',
     'firstName': 'Mike',
     'lastName': 'B',
     'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
      'suffix': '/VBVIR1QJEIQSY4WG.jpg'}},
    {'id': '7854066',
     'firstName': 'Tracy',
     'lastName': 'M',
     'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
      'suffix': '/OZQYYVRMJRMTUXUZ.jpg'}}]}],
 'summary': '3 Likes'}

### State park accessibility based on venues nearby
what are the other dimensions that might be interesting? <br>
closest shops/food; popularity; rating; activity